**Implementación de una Máquina de Estados Finito (FSM) Interactiva**

Este programa implementa un reconocedor de entradas de un usuario basado en una FSM predefinida. Requiere que el diseñador especifique la *tabla de transición*.

Primero, necesitamos instalar algunos paquetes:

In [1]:
%pip install spacy
!python -m spacy download es_core_news_sm

Note: you may need to restart the kernel to use updated packages.
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


Importamos algunas bibliotecas que necesitaremos:

In [2]:
import re
import es_core_news_sm
from unidecode import unidecode
import spacy
from spacy import displacy
from spacy.matcher import DependencyMatcher

In [3]:
# Settings

SlotFillings = {
    "ORIGEN": {"valor": "", "needQuestion": True, "pregunta": "¿Desde dónde quieres salir?"},
    "DESTINO": {"valor": "", "needQuestion": True, "pregunta": "¿A dónde quieres ir?"},
    "FECHA": {"valor": "", "needQuestion": True, "pregunta": "¿Qué día quieres salir?"},
    "HORA": {"valor": "", "needQuestion": True, "pregunta": "¿A qué hora quieres salir?"},
    "IDA/VUELTA": {"valor": "", "needQuestion": True, "pregunta": "¿Quieres ida y vuelta?"},
    "FECHA_VUELTA": {"valor": "", "needQuestion": True, "pregunta": "¿Qué día quieres volver?"},
    "HORA_VUELTA": {"valor": "", "needQuestion": True, "pregunta": "¿A qué hora quieres volver?"}
}

_ERRORQUESTION = "Lo siento, no te he entendido. ¿Podrías repetirlo?"
_WELCOMEQUESTION = "Hola, ¿en qué puedo ayudarte?"
_LOGOQUESTION = """                                _____
                       __...---'-----`---...__
                  _===============================
 ______________,/'      `---..._______...---'
(____________LL). .    ,--'
 /    /.---'       `. /
'--------_  - - - - _/
          `~~~~~~~~'"""

nlp = spacy.load('es_core_news_sm')

def matcherPattern(nlp):
    dep_matcher = DependencyMatcher(nlp.vocab)
    patterns = [
        [
            {"RIGHT_ID": "origen", "RIGHT_ATTRS": {"ENT_TYPE": {"IN": ["LOC", "GPE"]}}},
            {"LEFT_ID": "origen", "REL_OP": ">", "RIGHT_ID": "desde", "RIGHT_ATTRS": {"LOWER": "desde"}}
        ],
        [
            {"RIGHT_ID": "origen", "RIGHT_ATTRS": {"ENT_TYPE": {"IN": ["LOC", "GPE"]}}},
            {"LEFT_ID": "origen", "REL_OP": ">", "RIGHT_ID": "de", "RIGHT_ATTRS": {"LOWER": "de"}}
        ],
        [
            {"RIGHT_ID": "destino", "RIGHT_ATTRS": {"ENT_TYPE": {"IN": ["LOC", "GPE"]}}},
            {"LEFT_ID": "destino", "REL_OP": ">", "RIGHT_ID": "para", "RIGHT_ATTRS": {"LOWER": "para"}}
        ],
        [
            {"RIGHT_ID": "destino", "RIGHT_ATTRS": {"ENT_TYPE": {"IN": ["LOC", "GPE"]}}},
            {"LEFT_ID": "destino", "REL_OP": ">", "RIGHT_ID": "a", "RIGHT_ATTRS": {"LOWER": "a"}}
        ]
    ]

    for i, pattern in enumerate(patterns):
        dep_matcher.add(str(i), [pattern])

    return dep_matcher

dep_matcher = matcherPattern(nlp)

In [27]:
class Inicio:
    @staticmethod
    def run():
        print(_WELCOMEQUESTION)
        print(_LOGOQUESTION)
        print("¿En qué puedo ayudar?")
        return LeerRespuesta.run()

class LeerRespuesta:
    @staticmethod
    def run():
        try:
            respuestaUsuario = input()
            #respuestaUsuario = unidecode(respuestaUsuario)
            ProcesarRespuesta.run(respuestaUsuario)
        except:
            print(_ERRORQUESTION)
            #return LeerRespuesta.run()
        
class ProcesarRespuesta:
    @staticmethod
    def run(respuestaUsuario):
        doc = nlp(respuestaUsuario)
        #"proceso el texto en busca de coincidencias de sigma"
        ProcesarRespuesta.LeerMatcher(doc)
        ProcesarRespuesta.dibujarEntyToken(doc) #dibujo para entender las relaciones


    @staticmethod
    def LeerMatcher(doc):
        matches = dep_matcher(doc)
        for match_id, token_ids in matches:
            string_id = nlp.vocab.strings[match_id]  # Obtener la representación de cadena del ID del matcher
            matched_tokens = [doc[token_id] for token_id in token_ids]
            print(f"{string_id} encontró: {matched_tokens}")

    def dibujarEntyToken(doc):
        for token in doc:
            print(token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children])

        for ent in doc.ents:
            print(ent.text, ent.label_)
            
        displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})
        displacy.render(doc, style="ent", jupyter=True)

In [ ]:
class VerificarSlotFilling:
    @staticmethod
    def run(respuestaUsuario):
        #El sistema rellena todos los slot relevantes y continua con la conversación para llenar los slots restantes.
        return "miu"

In [ ]:
class SistemaDeDialogo:
    @staticmethod
    def run():
        return Inicio.run()

In [26]:
Inicio().run()

Hola, ¿en qué puedo ayudarte?
                                _____
                       __...---'-----`---...__
                  _===============================
 ______________,/'      `---..._______...---'
(____________LL). .    ,--'
 /    /.---'       `. /
'--------_  - - - - _/
          `~~~~~~~~'
¿En qué puedo ayudar?
Lo siento, no te he entendido. ¿Podrías repetirlo?


In [ ]:
# @IN array oracion
# @OUT String Lemas (concatenados)
def Lematizar(oracion):
   doc = nlp(oracion)
   lemas = [token.lemma_ for token in doc]
   return(" ".join(lemas))